In [1]:
import torch
import torchvision
from torchvision import models
from PIL import Image
import pyheif
import os
import matplotlib.pyplot as plt

In [2]:
from torchvision import transforms

transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [5]:
!wget https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt

--2021-05-06 12:17:36--  https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21674 (21K) [text/plain]
Saving to: ‘imagenet_classes.txt’

100%[======================================>] 21,674      --.-K/s   in 0s      

2021-05-06 12:17:36 (60.8 MB/s) - ‘imagenet_classes.txt’ saved [21674/21674]



In [7]:
machine_path = os.getcwd()
user_dir = '/'.join(machine_path.split('/')[:-1])

In [9]:
data_dir = "%s/data"%(user_dir)

In [11]:
!pip install av

     |████████████████████████████████| 38.1 MB 4.7 kB/s ta 0:00:0101    |████████████▏                   | 14.5 MB 8.1 MB/s eta 0:00:03     |███████████████                 | 17.9 MB 19.4 MB/s eta 0:00:02


In [27]:
os.mkdir('../data/mouse')

FileExistsError: [Errno 17] File exists: '../data/mouse'

In [29]:
import av

container = av.open('../data/IMG_0590.MOV')

packet_num = 0
for packet in container.demux():
    packet_num += 1
    for frame in packet.decode():
        if type(frame) != av.audio.frame.AudioFrame:
            img = frame.to_image()
            frame.to_image().save(('../data/mouse/packet_%s_frame_%04d.jpg'%(packet_num, frame.index)))

In [30]:
def read_heif(impath):
    heif_file = pyheif.read(impath)
    image = Image.frombytes(
        heif_file.mode, 
        heif_file.size, 
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
        )
    return image

In [31]:
model = models.resnet18(pretrained=True)
model.eval();

In [32]:
with open('imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]

In [33]:
def infer_image(impath, show = False, heif = True):
    if heif:
        img = read_heif(impath)
    else:
        img = Image.open(impath)
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    out = model(batch_t)
    
    _, indices = torch.sort(out, descending=True)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    title = [(classes[idx], percentage[idx].item()) for idx in indices[0][:1]]
    
    if show:
        plt.imshow(img)
        plt.title('Prediction:%s'%title)
        plt.axis('off')
        plt.show()
    else:
        print(title)

In [34]:
coffee_cup_folder = '/net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/spandan/differentiable_graphics_ml/data/mouse/'


In [35]:
len(os.listdir(coffee_cup_folder))

674

In [36]:
# os.listdir(coffee_cup_folder)

In [25]:
for path in os.listdir(coffee_cup_folder):
    if 'ipynb' not in path:
        imgpath = "%s/%s"%(coffee_cup_folder, path)
        infer_image(imgpath, show = True, heif = False)

UnidentifiedImageError: cannot identify image file '/net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/spandan/differentiable_graphics_ml/data/bottle//IMG_0552.HEIC'